In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

## Loading Environment File

In [ ]:
# Try loading from environment first, then from dev.env file
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    secret_path = Path("../.env")
    print("Looking for dev.env at:", secret_path.resolve())
    if secret_path.exists():
        load_dotenv(secret_path)
        GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in environment variables or dev.env file.")

print("GOOGLE_API_KEY loaded:", bool(GOOGLE_API_KEY))

## Setup: Imports, Retry Config

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

In [ ]:
import sys

# Ensure the project root (the parent of the "src" directory) is on sys.path
# so that "import src.model" finds the src package under the project root.
project_root = Path.cwd().parent
src_dir = project_root / "src"

project_root_path = str(project_root.resolve())
if project_root_path not in sys.path:
    sys.path.insert(0, project_root_path)

from src.model import Intensity, SentimentOutput
from src.model.rag_output import RagOutput, SuccessResponse, ErrorResponse

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agents Creation

In [ ]:
rag_output = RagOutput(
    result=SuccessResponse(
        response="This is a sample response from the RAG output model."
    )
)

sentiment_output = SentimentOutput(
    frustration_level=Intensity.LOW,
    neutrality_level=Intensity.LOW,
    satisfaction_level=Intensity.HIGH,
    confusion_level=Intensity.MEDIUM
)

prompt = f"""Combine these findings into a single executive summary:

    **RAG output:**
    {rag_output}
    
    **User sentiment analysis:**
    {sentiment_output}
"""

In [ ]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction=prompt,
    output_key="final_output",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

## Engine Def

Check the instructions:

In [ ]:
aggregator_agent.instruction

In [ ]:
runner = InMemoryRunner(agent=aggregator_agent)

## Testing

In [ ]:
response = await runner.run_debug(
    "Should I vaccinate?"
)

# TODOs

- benchmark della performance su casi limite della sentiment analysis
- valutare cosa fare per gestire i casi limite. Aggiungere un tool con un human-in-the-loop bloccante? Aggiungere un tool per loggare su un DB esterno il messaggio flaggato?